In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.1 MB 13.5 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
     |████████████████████████████████| 3.3 MB 30.5 MB/s 
     |████████████████████████████████| 596 kB 33.1 MB/s 
     |████████████████████████████████| 1.2 MB 16.1 MB/s 


In [2]:
# Mount Google drive to upload datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The path to the data on my drive
D = '/content/drive/My Drive/W266_Project_Data/pmi_data'

In [4]:
import pandas as pd
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [5]:
# Load the gold-standard target translation datasets
hi_target = pd.read_csv(D+"/predicted_text/MBart/dev.hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ml_target = pd.read_csv(D+"/predicted_text/MBart/dev.ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ta_target = pd.read_csv(D+"/predicted_text/MBart/dev.ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])

In [6]:
hi_target.head()

,sentence_1
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [7]:
# Load the Bart translated files into their respective language datasets
hi_bart = pd.read_csv(D+"/predicted_text/MBart/dev_bart_translated_hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])
ml_bart = pd.read_csv(D+"/predicted_text/MBart/dev_bart_translated_ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])
ta_bart = pd.read_csv(D+"/predicted_text/MBart/dev_bart_translated_ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])

In [8]:
hi_bart.head()

,sentence_2
0,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...
1,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [9]:
# Create Hindi Bart test set
hi_bart_test = pd.concat([hi_target['sentence_1'], hi_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
hi_bart_test.insert(0, 'language_task', 'translate English to Hindi')
hi_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [10]:
# Create Malayalam Bart test set
ml_bart_test = pd.concat([ml_target['sentence_1'], ml_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ml_bart_test.insert(0, 'language_task', 'translate English to Malayalam')

In [11]:
# View first few rows of Malayalam Bart test set
ml_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില് അഭിലാഷങ്ങള് സൃ...
1,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,രാജ്യത്തിന്റെ വികസയാത്രയില് പിന്നോക്കം വിട്ടുപ...
2,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ലക്ഷ്യം പൂര് ത്തിയാക്കും.
3,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,സമയബന്ധമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടെയും ഗവണ്മെന...
4,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ജന് ധന് അക്കൗണ്ട് തുറക്കാന്, ദരിദ്രര് ക്ക് ഇന്..."


In [12]:
# Load the first few rows of the Tamil Bart test set
ta_bart_test = pd.concat([ta_target['sentence_1'], ta_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ta_bart_test.insert(0, 'language_task', 'translate English to Tamil')

In [13]:
# View the first few rows of the Tamil translation
ta_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Tamil,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,கோடிக்கணக்கான மக்களின் மனங்களிலும் இதயங்களிலும...
1,translate English to Tamil,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...
2,translate English to Tamil,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022 ஆம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ...
3,translate English to Tamil,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,மத்திய அரசு பல்வேறு திட்டங்களை தெளிவான இலக்குக...
4,translate English to Tamil,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","ஜன் தன் கணக்குகள், ஏழை மக்களுக்கான காப்பீடு, உ..."


In [14]:
class SiameseNet(torch.nn.Module):
    def __init__(self):
      super(SiameseNet, self).__init__()

      # Set up a fully-connected layer to apply to each incoming sentence vector
      # Use a dense layer (nn.Linear) to reduce the dimensionality of the input embeddings
      # use a relu activation function
      # Randomly drop out
      # Set 1 as the dimensionality of the final output since we want to predict the logits corresponding to a binary label
      # Initialize the feed forward layer
      self.fc = torch.nn.Sequential(
            torch.nn.Linear(768, 128),
            torch.nn.InstanceNorm1d(128),
            torch.nn.ReLU(inplace=True))
      
      self.relu = torch.nn.ReLU(inplace=True)
      self.batch2 = torch.nn.InstanceNorm1d(384)
      self.ffn = torch.nn.Linear(384, 128)
      self.batch3 = torch.nn.InstanceNorm1d(128)
      self.layer_out = torch.nn.Linear(128,1)

      # Add drop out
      self.drop = torch.nn.Dropout(0.2)

    # left and right represent columns in our data
    def forward(self, left, right):

      # Get the sentence embeddings 
      u = left
      v = right

      # Reduce the dimensionality
      u = self.fc(u)
      v = self.fc(v)

      # Get the absolute value of the difference of the vectors
      uv = torch.sub(u, v)
      uv_abs = torch.abs(uv)

      # concatenate u, v, and the absolute value of the element-wise difference between the embeddings
      x = torch.cat([u, v, uv_abs], dim=-1)

      # Normalize the concatenated batch
      x = self.batch2(x)

      # Use the relu activation function
      x = self.relu(x)

      # Pass the concatenated layers through a dense layer to reduce the dimensionality from 2304 to 128
      x = self.ffn(x)
      # Normalize the concatenated layers
      x = self.batch3(x)
      # Pass through relu activation
      x = self.relu(x)
      # Implement drop out to avoid overfitting
      x = self.drop(x)
      # Pass the output through a final dense layer
      output = self.layer_out(x)
      # Reshape the output to have the same dimension as the target labels
      output = output[:,-1,:]

      # Concatenate the 
      return output

In [15]:
# Set up the device to run on GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [16]:
# Create an instance of the tokenizer and model using Indic Bert
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', keep_accents=True)
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def sentence_embeddings(device, tokenizer, model, sentences):
  """Create contextualized sentence embeddings
  
  Parameters:
  tokenizer: transformers tokenizer for creating word tokens
  model:
  sentences: sentences to generate sentence embeddings for
  """

  # Tokenize sentences and return PyTorch tensors
  encoded_input = tokenizer(sentences, padding=True, truncation=True, 
                            max_length=128, return_tensors='pt')
  encoded_input = encoded_input.to(device)
  
  # Pass the tokenized input to the model 
  with torch.no_grad():
    model.to(device)
    model_output = model(**encoded_input) 
  
  # Perform mean pooling to get total sentence embeddings
  sentence_embeddings = mean_pooling(model_output, 
                                     encoded_input['attention_mask'])
  
  return sentence_embeddings

In [19]:
# Load the Siamese Neural Net from the pre-trained weights
net = SiameseNet().to(device)
# Load the weights and biases from the last round of training
net.load_state_dict(torch.load("/content/drive/My Drive/W266_Project_Data/models/siamese_model/three_dense_w_punjabi_model.pt"))

<All keys matched successfully>

In [20]:
# Create a dataset class, similarly to the one used for training the Siamese Neural Network dataset - the difference is that it won't create labels
class TranslationEvalDataset(Dataset): 
  def __init__(self,training_df, device, tokenizer, model):
    self.training_df=training_df
    self.sent1 = self.training_df['sentence_1'].tolist()
    self.sent2 = self.training_df['sentence_2'].tolist()
    self.language_task = self.training_df['language_task'].tolist()
    self.device = device
    self.tokenizer = tokenizer
    self.model = model

  def __getitem__(self, index):
    sent1 = self.sent1[index]
    sent2 = self.sent2[index]
    task = self.language_task[index]

    # Get the sentence embeddings for sentences 1 and 2 for each row
    sent1_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent1)
    sent2_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent2)
    return sent1_embeddings, sent2_embeddings, task

  def __len__(self):
    return len(self.training_df)

In [21]:
# Concatenate all of the bart test sets into one test set
final_test = pd.concat([hi_bart_test, ml_bart_test, ta_bart_test]).reset_index()
final_test.drop(columns={'index'}, inplace=True)

In [22]:
final_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [23]:
final_test[1000:2000]

,language_task,sentence_1,sentence_2
1000,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില് അഭിലാഷങ്ങള് സൃ...
1001,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,രാജ്യത്തിന്റെ വികസയാത്രയില് പിന്നോക്കം വിട്ടുപ...
1002,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ലക്ഷ്യം പൂര് ത്തിയാക്കും.
1003,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,സമയബന്ധമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടെയും ഗവണ്മെന...
1004,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ജന് ധന് അക്കൗണ്ട് തുറക്കാന്, ദരിദ്രര് ക്ക് ഇന്..."
...,...,...,...
1995,translate English to Malayalam,മ്യാന്‍മാറിലെ യൂണിയന്‍ തിരഞ്ഞെടുപ്പ് കമ്മിഷന്‍ 7.,യൂണിയന് ရွေးကောက်ပွဲ് കമ്മിഷന് 7.
1996,translate English to Malayalam,സ്വാതന്ത്ര്യാനന്തരം പല നേട്ടങ്ങളും ഉണ്ടാക്കാന്...,സ്വതന്ത്രനെന്നതരം പല നേട്ടങ്ങള് ഉണ്ടാക്കാന് കഴ...
1997,translate English to Malayalam,"ജന്‍ ഭാഗീദാരി, വിവരം പങ്കിടല്‍, വിഭവങ്ങള്‍ പരമ...","ജാന് ഭാഗീദാരി, വിവരങ്ങള് പങ്കുവയ്ക്കാന്, വിഭവങ..."
1998,translate English to Malayalam,"ഗ്രാമ സ്വരാജ് അഭിയാന്‍, ആയുഷ്മാന്‍ ഭാരത് തുടങ്...","ഗ്രാമ സ്വ Raj അഭിയാന്, ആയുഷ്മാന് ഇന്ത്യ് തുടങ്..."


In [24]:
final_test.tail()

,language_task,sentence_1,sentence_2
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,6. மியான்மர் மத்திய தேர்தல் ஆணையம்
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...","பொது மக்களின் பங்களிப்பு, தகவல் ஓட்ட முறை, வளங..."
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","ஊரக சுயாட்சி திட்டம்,வளமான இந்தியா போன்ற சமீபத..."
2999,translate English to Tamil,இந்தியாவின் 130 லட்சம் மக்கள் சார்பில் உங்கள் ...,130 லட்சம் இந்தியர்களுக்காக புதுதில்லிக்கு உங்...


In [25]:
# Create an instance of the test set with all test translations
bart_test = TranslationEvalDataset(final_test, device, tokenizer, model)

In [26]:
# Create an instance of the test data loader
bart_test_loader = DataLoader(bart_test, batch_size=16)

In [27]:
# Track the language_tasks and sentences  being evaluated
y_language_task = final_test.language_task.tolist()
y_sent1 = final_test.sentence_1.tolist()
y_sent2 = final_test.sentence_2.tolist()
probability_of_paraphrase = []

with torch.no_grad():

  # Place the model into evaluation mode
    net.eval()

    # Loop over the test set
    for i, data in enumerate(bart_test_loader, 0):

      # Extract the test set components
      sent1_test, sent2_test, task_test = data

      # Send the test data to GPU
      sent1_test, sent2_test = sent1_test.to(device), sent2_test.to(device)

      # Get the test set predictions
      test_pred = net(sent1_test, sent2_test)
            
      # Get the predicted test labels
      y_test_pred = torch.sigmoid(test_pred).float()

      # Add the test labels to their respective list
      probability_of_paraphrase.extend(y_test_pred)

In [28]:
# Create a dataframe of the tasks, sentences, and probabilities of the second sentence (the translated one) being a paraphrase
y_probs = [x.item() for x in probability_of_paraphrase]
bart_performance = pd.DataFrame(list(zip(y_language_task, y_sent1, y_sent2, y_probs)),columns =['language_task', 'target_sentence', 'bart_translation', 'probability_of_paraphrase'])

In [29]:
# Check the first few rows
bart_performance.head()

,language_task,target_sentence,bart_translation,probability_of_paraphrase
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...,0.992026
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...,0.990891
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990999
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...",0.986538


In [30]:
# Sort the dataframe in order of lowest predicted probability and see where BART returns the lowest scores
# The lowest quality translations, as measured by paraphrase score, are those of Malayalam and Tamil
bart_performance.sort_values("probability_of_paraphrase")

,language_task,target_sentence,bart_translation,probability_of_paraphrase
2704,translate English to Tamil,"உழைப்பாளர்கள் தினம் என்று பேசும் போது, உழைப்பு...","‘தொழிலாளர்கள் தினத்தைக் கொண்டாடும் போது, ‘தொழி...",0.010269
1595,translate English to Malayalam,ഇന്ന് വിവിധ വിഷയങ്ങളെക്കുറിച്ചും മുന്നോട്ടുള്ള...,ഇന്ന് ഞങ്ങള് പല വിഷയങ്ങളും മുന്നോട്ടു പോകുന്ന ...,0.010438
1773,translate English to Malayalam,"സന്തോഷത്തിന്റെയും, ഐക്യത്തിന്റെയും മാനസിക ഭാവം...",സന്തോഷത്തിന് റെയും ഐക്യത്തിന് റെയും മാനസികഭാവം...,0.011403
1221,translate English to Malayalam,ഡിജിറ്റല്‍ വിപ്ലവത്തോടെ രാജ്യത്തെ ശക്തിപ്പെടുത...,പ്രസിഡന് റ് പ്രസിഡന് റ് പ്രിന് റ് പ്രസിഡന് റ് ...,0.012170
2483,translate English to Tamil,பழம்பெரும் ஹாக்கி வீரர் மேஜர் தியான் சந்துக்கு...,பாலின விகிதம் விகிதம் விகிதாசாரம் விகிதாசாரம் ...,0.012524
...,...,...,...,...
94,translate English to Hindi,वियतनाम यात्रा के दौरान मेरा प्रयास होगा कि लो...,वियतनाम यात्रा के दौरान मेरा प्रयास होगा कि लो...,0.992518
36,translate English to Hindi,"वे भारतीय प्रौद्योगिकी संस्थान, बम्बई के 56वें...","वे भारतीय प्रौद्योगिकी संस्थान, बम्बई के 56वें...",0.992550
100,translate English to Hindi,"इसी प्रकार, हम पिछले दो वर्षों में वैश्विक प्र...","इसी प्रकार, हमने पिछले दो वर्षों में वैश्विक प...",0.992577
382,translate English to Hindi,मैं इसे अच्छी निशानी मानता हूँ।,"मैं इसे अच्छी निशानी मानता हूँ। """,0.992815


In [31]:
# Check the number of rows with a predicted probability <0.5 (meaning that they would be assigned a label of 0 - not a paraphrase)
len(bart_performance[bart_performance.probability_of_paraphrase < 0.5])

532

In [32]:
# Check the number of translations of each language where the translation is not counted as a paraphrase
hi_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Hindi')])
ml_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Malayalam')])
ta_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Tamil')])
print(hi_errors)
print(ml_errors)
print(ta_errors)

6
317
209


In [33]:
# The comparatively high error rate for Malayalam and Tamil reflect their comparatively lower Sacre Bleu scores
hi_error_rate = (hi_errors/1000)
ml_error_rate = (ml_errors/1000)
ta_error_rate = (ta_errors/1000)

In [34]:
# Hindi performs very well
print("{0:.2%}".format(hi_error_rate))

0.60%


In [35]:
# Malayalam error rate
print("{0:.2%}".format(ml_error_rate))

31.70%


In [36]:
# The Tamil error rate is high, but not as high as Malayalam
print("{0:.2%}".format(ta_error_rate))

20.90%


In [40]:
# Get the sentence embeddings for a pair of "dissimilar sentences"
sent1_embed = sentence_embeddings(device, tokenizer, model, bart_performance['target_sentence'][2704])
sent2_embed = sentence_embeddings(device, tokenizer, model, bart_performance['bart_translation'][2704])

In [41]:
# Reshape to calculate the cosine similarity
sent1_embed.squeeze(0).shape

torch.Size([768])

In [42]:
# Find the cosine similarity score between these pair of "dissimilar sentences"
cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

print(cosine_similarity_value)

tensor(0.5459, device='cuda:0')


In [43]:
# Try for another pair of sentences, this time, for those with a high similarity score
sent3_embed = sentence_embeddings(device, tokenizer, model, bart_performance['target_sentence'][1])
sent3_embed = sentence_embeddings(device, tokenizer, model, bart_performance['bart_translation'][1])

In [44]:
# The sentences are the same
F.cosine_similarity(sent3_embed.squeeze(0), sent3_embed.squeeze(0), dim=0)

tensor(1., device='cuda:0')

In [45]:
# Get the sentence embeddings and cosine similarity scores between each pair of sentences in the dataset
# This is to check if low scoring sentences have exact similarity scores or if they are of low quality
def check_cosine_similarity(df, device, tokenizer, model):

  # Create an empty list to check cosine similarity scores
  cosine_scores = []

  for index, rows in df.iterrows():
    # Get the sentence embeddings of each pair of target sentences
    sent1_embed = sentence_embeddings(device, tokenizer, model, rows['target_sentence'])
    sent2_embed = sentence_embeddings(device, tokenizer, model, rows['bart_translation'])

    # Reshape the embeddings to be of a single dimension 
    # Get the cosine similarity between the two pairs of sentence embeddings
    cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

    # Add the cosine similarity score to the list of scores
    cosine_scores.append(cosine_similarity_value.item())

  return cosine_scores

In [46]:
translation_scores = check_cosine_similarity(bart_performance, device, tokenizer, model)

In [47]:
# Add the translation similarity scores to the indictrans performance dataframe
bart_performance['cosine_similarity_scores'] = translation_scores

In [48]:
# Sentneces with low cosine similarity also have low probability of paraphrase
bart_performance.sort_values('cosine_similarity_scores')

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
2961,translate English to Tamil,சுவீடன் அரசர் மாட்சிமை பொருந்திய கார்ல் பதினாற...,பத்ம விபூரண விபூரண விபூரண விபூரண விபூரண விபூரண...,0.016174,0.286790
2040,translate English to Tamil,"நம்மை எதிரிகளிடமிருந்தும், இயற்கைச் சீற்றம் மற...","இத்தகைய விலங்குகள், விலங்குகள், விலங்குகள், வி...",0.013086,0.293845
1292,translate English to Malayalam,സഹകരണവും ആശയവിനിമയവും ആവശ്യമുള്ള എല്ലാ വിഷയങ്ങ...,കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ...,0.017561,0.361691
2930,translate English to Tamil,புராதனமானது என்பதாலேயே ஒரு கருத்தை மறுதலிப்பது...,"குறிப்பாக, குறிப்பாக, குறிப்பாக, குறிப்பாக, கு...",0.012812,0.371510
2390,translate English to Tamil,சிவாஜி மகராஜ் அனைத்திற்கும் மேலாக தனது மக்களின...,பத்மசிறீவியின் மகனான பத்மசிறீவின் மகனான பத்மசி...,0.096139,0.384583
...,...,...,...,...,...
574,translate English to Hindi,"वेतनभोगी, मध्यम वर्ग को दी गई टैक्स राहत के लि...","वेतनभोगी, मध्यम वर्ग को दी गई टैक्स राहत के लि...",0.988739,1.000000
575,translate English to Hindi,ये बजट हर भारतीय की आशाओं-आकांक्षाओ पर खरा उतर...,ये बजट हर भारतीय की आशाओं-आकांक्षाओ पर खरा उतर...,0.990355,1.000000
576,translate English to Hindi,"प्रधानमंत्री ने कहा, ‘मैं नेताजी सुभाष चन्द्र ...","प्रधानमंत्री ने कहा, ‘मैं नेताजी सुभाष चन्द्र ...",0.987992,1.000000
2262,translate English to Tamil,ஜப்பானுடனான நமது உறவு சிறப்பான உத்தி நிறைந்த ச...,ஜப்பானுடனான நமது உறவு சிறப்பான உத்தி நிறைந்த ச...,0.984359,1.000000


In [49]:
# High similarity scores have a high probability of paraphrase
bart_performance.sort_values('cosine_similarity_scores', ascending=False)

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
338,translate English to Hindi,प्रधानमंत्री इंदौर में आयोजित होने वाले शहरी व...,प्रधानमंत्री इंदौर में आयोजित होने वाले शहरी व...,0.991518,1.000000
410,translate English to Hindi,"भारत का तेजी से विकास पूरे क्षेत्र, विशेषकर श्...","भारत का तेजी से विकास पूरे क्षेत्र, विशेषकर श्...",0.990341,1.000000
407,translate English to Hindi,"हमारी शक्ति हमारे ज्ञान, क्षमता और समृद्धि को ...","हमारी शक्ति हमारे ज्ञान, क्षमता और समृद्धि को ...",0.991411,1.000000
2853,translate English to Tamil,பொதுக்கூட்டத்திலும் அவர் உரையாற்றுவார்.,பொதுக்கூட்டத்திலும் அவர் உரையாற்றுவார்.,0.980024,1.000000
402,translate English to Hindi,मेरे तमिल भाई-बहन भी काशी विश्वनाथ की भूमि वार...,मेरे तमिल भाई-बहन भी काशी विश्वनाथ की भूमि वार...,0.990173,1.000000
...,...,...,...,...,...
2390,translate English to Tamil,சிவாஜி மகராஜ் அனைத்திற்கும் மேலாக தனது மக்களின...,பத்மசிறீவியின் மகனான பத்மசிறீவின் மகனான பத்மசி...,0.096139,0.384583
2930,translate English to Tamil,புராதனமானது என்பதாலேயே ஒரு கருத்தை மறுதலிப்பது...,"குறிப்பாக, குறிப்பாக, குறிப்பாக, குறிப்பாக, கு...",0.012812,0.371510
1292,translate English to Malayalam,സഹകരണവും ആശയവിനിമയവും ആവശ്യമുള്ള എല്ലാ വിഷയങ്ങ...,കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ...,0.017561,0.361691
2040,translate English to Tamil,"நம்மை எதிரிகளிடமிருந்தும், இயற்கைச் சீற்றம் மற...","இத்தகைய விலங்குகள், விலங்குகள், விலங்குகள், வி...",0.013086,0.293845


In [66]:
# Find the range in cosine similarity scores for sentences not considered paraphrases of each other
np = bart_performance[bart_performance.probability_of_paraphrase < 0.5].reset_index()
np.drop(columns={'index'}, inplace=True)

In [67]:
# Find the min and max of the cosine similarity scores
print(min(np.cosine_similarity_scores))
# There are some sentences labeled as not paraphrases of each other that have a high degree of cosine similarity
print(max(np.cosine_similarity_scores))

0.28679001331329346
0.9831867218017578


In [68]:
# Do the same for sentences considered paraphrases of each other
p = bart_performance[bart_performance.probability_of_paraphrase >= 0.5]
print(min(p.cosine_similarity_scores))
print(max(p.cosine_similarity_scores))

0.4736849367618561
1.0


In [69]:
# Look at the pair of sentences with the highest degree of cosine similarity
np[np['cosine_similarity_scores'] > 0.5]

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
0,translate English to Hindi,प्रधानमंत्री ने विदेशों में रहने वाले भारतीय न...,प्रधानमंत्री ने विदेश मंत्री श्रीमती सुषमा स्व...,0.462450,0.853695
1,translate English to Hindi,मेरे प्यारे देशवासियो !,मेरे प्यारे देशवासियो!,0.248533,0.740558
2,translate English to Hindi,मैं कहता हूँ – इसमें बुराई क्या है ?,मैं कहता हूँ- इसमें बुराई क्या है?,0.432159,0.703669
3,translate English to Hindi,उन्होंने इस अवसर पर सांस्कृतिक प्रदर्शन में पर...,उन्होंने इस अवसर पर भारतीय समुदाय की सराहना की।,0.439216,0.875689
4,translate English to Hindi,उन्हें एक नये जोश और उत्साह से भर देगा।,और उन्हें एक नया उत्साह प्रदान करेगा।,0.297890,0.840880
...,...,...,...,...,...
527,translate English to Tamil,சமீபத்தில் துவங்கப்பட்ட அந்த்யோதயா விரைவு ரயில...,அண்மையில் தொடங்கப்பட்ட அந்தோதயா விரைவு ரயில் வ...,0.246689,0.963187
528,translate English to Tamil,“ஆசியான்-இந்தியா மற்றும் கிழக்கு ஆசிய உச்சி மா...,“அசியான்-இந்தியா மற்றும் கிழக்க ஆசிய உச்சி மாந...,0.432035,0.925320
529,translate English to Tamil,"இதில், மூன்று அரங்கங்கள் மற்றும் கணினி களஞ்சிய...","இவற்றில் மூன்று வானூர்தி நிலையங்களும், மூன்று ...",0.027963,0.761624
530,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","ஊரக சுயாட்சி திட்டம்,வளமான இந்தியா போன்ற சமீபத...",0.258094,0.977657


In [70]:
np.sort_values('probability_of_paraphrase', ascending=False)

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
139,translate English to Malayalam,ചിലപ്പോഴൊക്കെ നമ്മുടെ മുന്‍ധാരണകളാണ് ഒരു തുറന്...,ചിലപ്പോള് നമ്മുടെ മുന് ധാരണകള് ഒരു തുറന്ന ചര് ...,0.499189,0.932574
377,translate English to Tamil,நாட்டில் மேம்படுத்தப்பட்ட உள்கட்டமைப்பினால் நட...,"நாட்டில் மேம்படுத்தப்பட்ட உள்கட்டமைப்பு, நடுத்...",0.497797,0.948354
51,translate English to Malayalam,നിര്‍ദിഷ്ട ഭേദഗതികള്‍ സംബന്ധിച്ച അന്തിമതീരുമാന...,ഇത് ബില് നളന്ദ സര് വകലാശാലയിലെ പുതിയ ഭരണസമിതിയ...,0.496805,0.932117
209,translate English to Malayalam,പ്രകാശ്ജി അദ്ദേഹത്തിന്റെ നീണ്ട കത്ത് ആരംഭിച്ചി...,പ്രകാശ് ജി തന് റെ നീണ്ട കത്ത് തുടങ്ങുന്നത് കല്...,0.496552,0.889826
216,translate English to Malayalam,തമിഴ്‌നാട് രാഷ്ട്രീയരംഗത്തെ ദീര്‍ഘകാല പ്രവര്‍ത...,അദ്ദേഹം രാഷ്ട്രീയ ജീവിതത്തില് പൊതുസമൂഹത്തിലും ...,0.496156,0.907856
...,...,...,...,...,...
421,translate English to Tamil,பழம்பெரும் ஹாக்கி வீரர் மேஜர் தியான் சந்துக்கு...,பாலின விகிதம் விகிதம் விகிதாசாரம் விகிதாசாரம் ...,0.012524,0.591702
76,translate English to Malayalam,ഡിജിറ്റല്‍ വിപ്ലവത്തോടെ രാജ്യത്തെ ശക്തിപ്പെടുത...,പ്രസിഡന് റ് പ്രസിഡന് റ് പ്രിന് റ് പ്രസിഡന് റ് ...,0.012170,0.596631
247,translate English to Malayalam,"സന്തോഷത്തിന്റെയും, ഐക്യത്തിന്റെയും മാനസിക ഭാവം...",സന്തോഷത്തിന് റെയും ഐക്യത്തിന് റെയും മാനസികഭാവം...,0.011403,0.739193
194,translate English to Malayalam,ഇന്ന് വിവിധ വിഷയങ്ങളെക്കുറിച്ചും മുന്നോട്ടുള്ള...,ഇന്ന് ഞങ്ങള് പല വിഷയങ്ങളും മുന്നോട്ടു പോകുന്ന ...,0.010438,0.898440


In [73]:
# Get the target sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'target_sentence'].iloc[0]

'வாரணாசியில் அமைக்கப்பட்டுள்ள சர்வதேச அரிசி ஆராய்ச்சி நிலையம் பற்றி சுட்டிக்காட்டிய பிரதமர், தொழில்நுட்பத்தைப் பயன்படுத்தி, வேளாண் துறையை அதிக லாபம் உள்ள தொழிலாக மாற்ற, அரசு மேற்கொண்ட முயற்சிகளின் பலனாகவே இந்த மையம் ஏற்படுத்தப்பட்டுள்ளது என்றார்.'

In [74]:
# Get the translation sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'bart_translation'].iloc[0]

'வாரணாசியில் அமைக்கப்பட்டுள்ள சர்வதேச அரிசி ஆராய்ச்சி நிலையம், தொழில்நுட்பத்தை பயன்படுத்தி, விவசாயத்தை அதிகலாப தொழில்களாக மாற்றியமைத்து, அரசு மேற்கொண்ட முயற்சிகளின் பலமாக இந்த மையம் அமைக்கப்பட்டுள்ளது என்று பிரதமர் குறிப்பிட்டார்.'

In [56]:
# The English translation of the target sentence with the maximum cosine similarity score - using Google translate to translate to English
english_np_target = 'Referring to the International Rice Research Station in Varanasi, the Prime Minister said the center was set up as a result of the government\'s efforts to transform agriculture into a more lucrative industry using technology.'
english_np_translation = 'The Prime Minister said the center was set up as part of the government\'s efforts to transform agriculture into a lucrative industry using technology at the International Rice Research Station in Varanasi.'
# The target language was Tamil, which has the highest model error rate. As we can see below, this is an error of our model (the sentences are paraphrases)
print(english_np_target)
print(english_np_translation)

Referring to the International Rice Research Station in Varanasi, the Prime Minister said the center was set up as a result of the government's efforts to transform agriculture into a more lucrative industry using technology.
The Prime Minister said the center was set up as part of the government's efforts to transform agriculture into a lucrative industry using technology at the International Rice Research Station in Varanasi.


In [75]:
# Let's look at the sentence pair with the lowest cosine similarity in the paraphrase dataset
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'target_sentence'].iloc[0])
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'bart_translation'].iloc[0])

ഈ ദിശയില്‍ ഉഭയകക്ഷി വ്യാപാരവും സാമ്പത്തിക ബന്ധങ്ങളും ശക്തിപ്പെടുത്തുന്നതിനും രണ്ടു രാജ്യങ്ങളിലെയും ജനതകള്‍ തമ്മിലുള്ള വിനിമയങ്ങള്‍ വന്‍തോതില്‍ പ്രോല്‍സാഹിപ്പിക്കുന്നതിനും നാം പ്രവര്‍ത്തിക്കും.
In this direction we will strengthen trade and economic relations and largely stimulate the relations between the peoples of both countries.


In [76]:
# The output string was a failure to translate - BART left the sentence in English
# The English back translation is seen below (coped from Google translate)
# The sentence meanings are similar and our model recognized that even though the languages were different
back_trans = 'In this direction, we will work to strengthen bilateral trade and economic ties and to greatly promote trade between the peoples of the two countries.'

In [77]:
# Write the bart performance file to a csv 
bart_performance.to_csv(D+"/predicted_text/siamese_evaluations_of_translations/bart_siamese_paraphrase_eval.csv")